#현재 코드
## 해시태그 유사하면
## 카테고리 유사하면

# 고려할만한 요소들
## 지역 고려할 것인가? 지역의 유사성
## 조회수 높으면 (오름차순하는 용도)
## 현재 신청 중이면 (마감된 것도 넣으면 마감 된 거는 회색으로 처리한다든가..?)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Masil")
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/Colab Notebooks/Masil'

In [ ]:
import json
import csv


import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
with open("all_program.json", "r", encoding="utf8") as input_json:
    contents = input_json.read() # string 타입
    json_data = json.loads(contents)

print(json_data) # 전체 JSON을 dict type으로 가져옴

{'message': 'monthly programs has fetched successfully', 'data': {'count': 208, 'rows': [{'id': 234, 'programName': '「하동으로, 여행 가자!(경남에서 한 달 여행하기)」 (4차)', 'applyStartDate': '2023-10-23', 'applyEndDate': '2023-11-30', 'tourStartDate': '2023-10-23', 'tourEndDate': '2023-12-08', 'benefit': '숙박비 1일 50,000원\n체험비 1일 50,000원 (7박 미만) 또는 80,000원 (7박 이상)', 'provinceId': 16, 'cityId': 1256, 'posterUrl': '', 'url': 'https://www.hadong.go.kr/media/00012.web?amode=view&not_ancmt_mgt_no=36100&cpage=1&sstring=%ED%95%9C+%EB%8B%AC&stype=title', 'status': 'dday ing', 'hits': 97, 'isApplyDateInMonthFormat': False, 'isTourDateInMonthFormat': False, 'conditions': '공고일 기준 만19세 이상의 경남지역 외 거주자\n하동 여행에 대한 의지가 크고, 개인 SNS 등을 통해 지역 관광자원을 불특정다수에게 홍보가 가능한 자(SNS계정을 공개하여 활용하는 자)', 'etc': None, 'source': '하동군청', 'detail': '<div style="line-height: 150%;">\n<h3><span style="color: rgb(0, 0, 0);"><strong>상세 정보</strong></span></h3>\n<p style="color: rgb(51, 51, 51); line-height: 150%;"><span style="color: rgb(0, 0, 0);"><s

In [ ]:
with open("program_11_19.json", "r", encoding="utf8") as input_json:
    contents = input_json.read() # string 타입
    json_data = json.loads(contents)

print(json_data) # 전체 JSON을 dict type으로 가져옴

{'data': {'count': 140, 'rows': [{'id': 259, 'programName': '경북 웰니스 팸투어', 'categoryId': 1, 'categoryName': '살아보기', 'hits': 116, 'proviceId': 15, 'provinceName': '경상북도', 'cityId': None, 'hashtags': 'SNS 미션,짧은 프로그램,인플루언서'}, {'id': 258, 'programName': '도시청년 하동에서 살아보기', 'categoryId': 1, 'categoryName': '살아보기', 'hits': 150, 'proviceId': 16, 'provinceName': '경상남도', 'cityId': 1256, 'cityName': '하동군', 'hashtags': '교통비 지원,무료,숙박비 지원,식비 지원,청년 우대,짧은 프로그램'}, {'id': 249, 'programName': '경남에서 한 달 여행하기 사업 ‘Wellnessmate, 산청’ (4차)', 'categoryId': 1, 'categoryName': '살아보기', 'hits': 411, 'proviceId': 16, 'provinceName': '경상남도', 'cityId': 1257, 'cityName': '산청군', 'hashtags': '전 연령 가능,SNS 미션,체험비 지원,숙박비 지원,긴 프로그램,짧은 프로그램'}, {'id': 245, 'programName': '세컨하우스 살아보기 시범운영 참가자 모집 공고', 'categoryId': 1, 'categoryName': '살아보기', 'hits': 296, 'proviceId': 13, 'provinceName': '전라북도', 'cityId': 1338, 'cityName': '진안군', 'hashtags': '전 연령 가능,자비 부담'}, {'id': 244, 'programName': '상주 숨은여행찾기', 'categoryId': 1, 'categoryName': 

In [ ]:
print(json_data['data']['rows'])

[{'id': 259, 'programName': '경북 웰니스 팸투어', 'categoryId': 1, 'categoryName': '살아보기', 'hits': 116, 'proviceId': 15, 'provinceName': '경상북도', 'cityId': None, 'hashtags': 'SNS 미션,짧은 프로그램,인플루언서'}, {'id': 258, 'programName': '도시청년 하동에서 살아보기', 'categoryId': 1, 'categoryName': '살아보기', 'hits': 150, 'proviceId': 16, 'provinceName': '경상남도', 'cityId': 1256, 'cityName': '하동군', 'hashtags': '교통비 지원,무료,숙박비 지원,식비 지원,청년 우대,짧은 프로그램'}, {'id': 249, 'programName': '경남에서 한 달 여행하기 사업 ‘Wellnessmate, 산청’ (4차)', 'categoryId': 1, 'categoryName': '살아보기', 'hits': 411, 'proviceId': 16, 'provinceName': '경상남도', 'cityId': 1257, 'cityName': '산청군', 'hashtags': '전 연령 가능,SNS 미션,체험비 지원,숙박비 지원,긴 프로그램,짧은 프로그램'}, {'id': 245, 'programName': '세컨하우스 살아보기 시범운영 참가자 모집 공고', 'categoryId': 1, 'categoryName': '살아보기', 'hits': 296, 'proviceId': 13, 'provinceName': '전라북도', 'cityId': 1338, 'cityName': '진안군', 'hashtags': '전 연령 가능,자비 부담'}, {'id': 244, 'programName': '상주 숨은여행찾기', 'categoryId': 1, 'categoryName': '살아보기', 'hits': 210, 'proviceId'

In [ ]:
columns = list(json_data['data']['rows'][0].keys())
print(columns)

['id', 'programName', 'categoryId', 'categoryName', 'hits', 'proviceId', 'provinceName', 'cityId', 'hashtags']


In [ ]:
with open("program_11_19.json", "r", encoding="utf8") as input_json, open('blank.csv', 'w', newline = '') as output_csv:
    contents = input_json.read() # string 타입
    json_data = json.loads(contents)
    json_data = json_data['data']['rows']

    # CSV 파일에 작성할 열 이름 추출
    columns = list(json_data[0].keys())

    # CSV writer 생성
    csv_writer = csv.writer(output_csv)

    # CSV 파일의 헤더 작성
    csv_writer.writerow(columns)

    # 각 행의 데이터를 CSV 파일에 작성
    for row in json_data:
        # 각 열의 값을 가져와서, 값이 없으면 빈 문자열을 사용
        csv_row = [row.get(column, '') for column in columns]
        csv_writer.writerow(csv_row)

"    columns = list(json_data['data']['rows'][0].keys())\n\n    f = csv.writer(output_csv)\n    f.writerow(columns)\n    #f.writerow(['id', 'programName', 'categoryId','categoryName' 'hits', 'proviceId', 'provinceName', 'cityId', 'cityName', 'hashtags' ])\n\n    for column in columns:\n        if json_data[column] is not None:\n            city_name = city_info.get()\n            city_id = city_info.get('id')\n            province_id = city_info.get('provinceId')\n        else:\n            city_name = ''\n            city_id = ''\n            province_id = ''\n        f. writerow([datum['id'], datum['programName'], datum['categoryId'], datum['hits'], province_id, city_id, city_name])"

In [ ]:
# 데이터 불러오기
df = pd.read_csv('blank.csv')
df

,id,programName,categoryId,categoryName,hits,proviceId,provinceName,cityId,hashtags
0,259,경북 웰니스 팸투어,1,살아보기,116,15,경상북도,NaN,"SNS 미션,짧은 프로그램,인플루언서"
1,258,도시청년 하동에서 살아보기,1,살아보기,150,16,경상남도,1256.0,"교통비 지원,무료,숙박비 지원,식비 지원,청년 우대,짧은 프로그램"
2,249,"경남에서 한 달 여행하기 사업 ‘Wellnessmate, 산청’ (4차)",1,살아보기,411,16,경상남도,1257.0,"전 연령 가능,SNS 미션,체험비 지원,숙박비 지원,긴 프로그램,짧은 프로그램"
3,245,세컨하우스 살아보기 시범운영 참가자 모집 공고,1,살아보기,296,13,전라북도,1338.0,"전 연령 가능,자비 부담"
4,244,상주 숨은여행찾기,1,살아보기,210,15,경상북도,1280.0,"청년 우대,숙소 제공,짧은 프로그램"
...,...,...,...,...,...,...,...,...,...
135,11,창원에서 한달살이 (1차),1,살아보기,27,16,경상남도,1244.0,"SNS 미션,체험비 지원,짧은 프로그램"
136,10,"양구탐험대, 지정기부 발굴 프로젝트",1,살아보기,28,10,강원도,1240.0,"청년 우대,체험비 지원,SNS 미션"
137,9,영월에서 일주일 살기 Check in 영월,1,살아보기,29,10,강원도,1235.0,"짧은 프로그램,체험비 지원,우수혜택,SNS 미션"
138,8,삼척 한 달 여행하기,1,살아보기,29,10,강원도,1232.0,"체험비 지원,청년 우대,SNS 미션"
